# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!


### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [2]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [3]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database '\\dc1\userspace\kummer\bosssdbs-minibatch\OscillatingDroplet3D'.


In [4]:
OpenOrCreateDatabase(@"\\fdygitrunner\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\fdygitrunner\ValidationTests\OscillatingDroplet3D'.


In [5]:
wmg.Sessions

#0: OscillatingDroplet3D	J432k3_arm1_mode4_aP2*	11/01/2021 20:00:02	c025e47c...
#1: OscillatingDroplet3D	J432k3_arm1_mode2_aP1*	11/01/2021 19:36:06	c5c88e22...
#2: OscillatingDroplet3D	J432k3_arm1_mode2_aP0*	11/01/2021 19:36:06	ece5c538...
#3: OscillatingDroplet3D	J432k3_arm1_mode2_aP2*	11/01/2021 19:36:06	c2c08313...
#4: OscillatingDroplet3D	J432k3_arm1_mode4_aP0*	11/01/2021 19:36:07	bc918d99...
#5: OscillatingDroplet3D	J432k3_arm1_mode4_aP1*	11/01/2021 19:36:06	4e3d2799...
#6: OscillatingDroplet3D	J432k3_arm1_mode3_aP2*	11/01/2021 19:36:06	925309b2...
#7: OscillatingDroplet3D	J432k3_arm1_mode3_aP1*	11/01/2021 19:36:06	e78dc9ef...
#8: OscillatingDroplet3D	J432k3_arm1_mode3_aP0*	11/01/2021 19:36:06	370651e3...
#9: OscillatingDroplet3D	J432k3_arm0_mode4_aP2*	11/01/2021 19:30:57	209fe1bd...
#10: OscillatingDroplet3D	J432k3_arm0_mode4_aP1*	11/01/2021 19:24:22	6f496029...
#11: OscillatingDroplet3D	J432k3_arm0_mode4_aP0*	11/01/2021 19:24:04	8525c74a...
#12: OscillatingDroplet3D	J432k3_arm0_

In [6]:
var S1 = wmg.Sessions[0];

In [7]:
S1.Tags 

index,value
0,NotTerminated


In [8]:
//foreach(var kv in S1.KeysAndQueries)
//   Console.WriteLine($"{kv.Key} \t\t= {kv.Value}");

In [9]:
using System.IO;

In [10]:
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string TextFile) {

var TextFile2 = Path.Combine(Path.GetDirectoryName(TextFile), Path.GetFileNameWithoutExtension(TextFile) + "-Copy.txt");
File.Copy(TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(TextFile), Path.GetFileNameWithoutExtension(TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
          
          
       } else {
          int minus = P0.IndexOf("-");
          
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}

In [11]:
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(S1);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    //FileSanitizer(file);
}

In [12]:
S1.PrintSessionDirectory();

\\fdygitrunner\ValidationTests\OscillatingDroplet3D\sessions\c025e47c-8fcc-4964-a39c-65692d984d55


In [13]:
var tab = S1.ReadTabulatedTextFileAsDoubles("SphericalHarmonics-Sanitized.txt", '\t');

In [14]:
tab

(empty)

In [15]:
var plot = new Plot2Ddata();
var time = tab["time"];
foreach(var column in tab) {
   if(column.Key == "time")
      continue;
}

Unhandled exception: System.Collections.Generic.KeyNotFoundException: The given key 'time' was not present in the dictionary.
   at System.Collections.Generic.Dictionary`2.get_Item(TKey key)
   at Submission#17.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)